In [1]:
import numpy as np
from sympy import *
from math import floor
import time

In [2]:
#Useful functions for both Hypercubes and general Hamming graphs
def read_sets(filename):
    #Read in the sets of a file
    results = []
    with open(filename,'r') as f:
        for line in f.read().splitlines():
            results.append(line.split(','))
    return(results)

def make_linearEqns(A,z):
    #Takes matrix A and variable list z and converts it into a list of linear equations
    z = Matrix(z)
    A = Matrix(A)
    A = A.rref()[0]
    lin_fcns = A*z
    lin_fcns = [f for f in lin_fcns if f != 0]
    return(list(lin_fcns))

In [3]:
def create_polys(k,a):
    #Setup polynomial system for H_k,a without knowing what R is
    variable_string = 'z1'
    for i in range(1,k*a):
        variable_string = variable_string +',z{} '.format(i+1)
    z = var(variable_string)
    P = []
    f = 0
    for i in range(k):
        func = 0
        #func2 = 0
        for j in range(0,a):
            P.append(z[i*a+j]*(z[i*a+j]-1)*(z[i*a+j]+1)) #1st condition zi*(zi-1)*(zi+1)
            func= func + z[i*a+j]**2
            #func2 = func2 + z[i*a+j]
            f = f + z[i*a+j]**2 #f so that sum(zi**2) neq 0
        func = (2-func)*(func)
        P.append(func)
        #P.append(func2)
    
    fs = [f-2*i for i in range(1,k+1)]
    return(P,fs,z)

def make_matrix(R,k,a):
    #Converts list of one-hot encodings to the linear system
    temp = [r.flatten('F') for r in R]
    for i in range(k):
        added_row = np.zeros(k*a)
        for j in range(a):
            #This is the 2nd condition where sum(zi)_i*a+1^i*a+a = 0
            added_row[i*a+j] = 1
        temp.append(list(added_row))
    return(np.array(temp,dtype = int))

def check_resolving(R,k,a,alphabet = None):
    (P,fs,z) = create_polys(k,a)
    OH_encodedR = OneHot(R,k,a)
    A = make_matrix(OH_encodedR,k,a)
    lin_fcns = make_linearEqns(A,z)
    G = groebner(P+lin_fcns,order = 'lex')
    for fi in fs:
        Gi = groebner(list(G)+[fi],order = 'lex')
        if not (list(Gi) == [1]):
            return False
    return True

In [4]:
def OneHot(R,k,a,alphabet = None):
    #Converts list of strings to list of one-hot encodings
    if alphabet == None:
        temp = [str(i) for i in range(a)]
        alphabet = ''.join(temp)
    encodings = []
    for r in R:
        encoding = np.zeros((a,k))
        for i in range(k):
            for j,letter in enumerate(alphabet):
                if r[i] == letter:
                    encoding[j,i] = 1
        encodings.append(encoding)
    return(encodings)

def create_polys_improved(k,a):
    #Setup polynomial system for H_k,a without knowing what R is
    variable_string = 'z1'
    for i in range(1,k*a):
        variable_string = variable_string +',z{} '.format(i+1)
    z = var(variable_string)
    P = []
    for i in range(k):
        Pi = []
        func = 0
        #func2 = 0
        for j in range(0,a):
            Pi.append(z[i*a+j]*(z[i*a+j]-1)*(z[i*a+j]+1)) #1st condition zi*(zi-1)*(zi+1)
            func= func + z[i*a+j]**2
            #func2 = func2 + z[i*a+j]
        func = (2-func)*(func)
        Pi.append(func)
        #Pi.append(func2)
        P.append(Pi)
    squares = [zi**2 for zi in z]
    sum_squares = sum(squares)
    f = 1
    for i in range(1,k+1):
        f = f*(2*i - sum_squares)
    return(P,f,z)

def preprocess_groebner(P,z,k,a):
    G0 = groebner(P[0],order = 'lex')
    G = [g for g in list(G0)]
    for i in range(1,k):
        replacements = [(z[j],z[a*i+j]) for j in range(a)]
        for g in list(G0):
            G.append(g.subs(replacements))
    return(G)

def make_matrix_improved(R,k,a):
    #Converts list of one-hot encodings to the linear system
    temp = [r.flatten('F') for r in R]
    for i in range(k):
        added_row = np.zeros(k*a)
        for j in range(a):
            #This is the 2nd condition where sum(zi)_i*a+1^i*a+a = 0
            added_row[i*a+j] = 1
        temp.append(list(added_row))
    return(np.array(temp,dtype = int))

def check_resolving_improved(R,k,a,alphabet = None):
    (P,f,z) = create_polys_improved(k,a)
    OH_encodedR = OneHot(R,k,a)
    A = make_matrix(OH_encodedR,k,a)
    lin_fcns = make_linearEqns(A,z)
    pre_G = preprocess_groebner(P,z,k,a)
    print('Finished Precomputing')
    G = groebner(pre_G+lin_fcns+[f],order = 'lex')
    print('Finished Groebner')
    if list(G) != [1]:
        return False
    return True


In [5]:
#Using NON-IMPROVED ALGORITHM
sets = read_sets('Test_Sets/H_3_5.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = check_resolving(R,5,3)
    is_resolving.append(resolve)
    end = time.time()
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

Set 1 is resolving
Set 2 is resolving
Set 3 is resolving
Set 4 is not resolving
Set 5 is not resolving
Set 6 is not resolving
Set 7 is resolving
Set 8 is not resolving
Total Time: 1.30 seconds
Average time: 0.16 seconds


In [6]:
#Using IMPROVED ALGORITHM
sets = read_sets('Test_Sets/H_3_5.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = check_resolving_improved(R,5,3)
    is_resolving.append(resolve)
    end = time.time()
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

Finished Precomputing


KeyboardInterrupt: 

In [7]:
#Using NON-IMPROVED ALGORITHM
sets = read_sets('Test_Sets/H_4_4.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = check_resolving(R,4,4)
    is_resolving.append(resolve)
    end = time.time()
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

Set 1 is resolving
Set 2 is not resolving
Set 3 is not resolving
Set 4 is not resolving
Set 5 is resolving
Set 6 is resolving
Set 7 is resolving
Set 8 is not resolving
Total Time: 2.25 seconds
Average time: 0.28 seconds


In [45]:
#Using IMPROVED ALGORITHM
sets = read_sets('Test_Sets/H_4_4.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = check_resolving_improved(R,4,4)
    is_resolving.append(resolve)
    end = time.time()
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

Set 1 is resolving
Set 2 is not resolving
Set 3 is not resolving
Set 4 is not resolving
Set 5 is resolving
Set 6 is resolving
Set 7 is resolving
Set 8 is not resolving
Total Time: 3.78 seconds
Average time: 0.47 seconds


In [46]:
#Using NON-IMPROVED ALGORITHM
sets = read_sets('Test_Sets/H_4_5.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = check_resolving(R,5,4)
    is_resolving.append(resolve)
    end = time.time()
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))

KeyboardInterrupt: 

In [ ]:
#Using IMPROVED ALGORITHM
sets = read_sets('Test_Sets/H_4_5.txt')
is_resolving = []
time_total = 0
num_sets = 0
for i,R in enumerate(sets):
    start = time.time()
    resolve = check_resolving_improved(R,5,4)
    is_resolving.append(resolve)
    end = time.time()
    time_total = time_total + end-start
    num_sets = num_sets+1
avg_time = time_total/num_sets
for i,r in enumerate(is_resolving):
    if r:
        print('Set {} is resolving'.format(i+1))
    else:
        print('Set {} is not resolving'.format(i+1))
print('Total Time: {:.2f} seconds'.format(time_total))
print('Average time: {:.2f} seconds'.format(avg_time))